# **DATA INGESTION**

In [2]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [5]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 5.6 MB/s eta 0:00:00


In [15]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

In [16]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory."""
    all_documents = []
    pdf_dir = Path(pdf_directory)

    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))

    print(f"Found {len(pdf_files)} PDF files to process")

    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()

            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'

            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")

        except Exception as e:
            print(f"  ✗ Error: {e}")

    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents


# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("/content/drive/MyDrive/Rag_Data")

Found 6 PDF files to process

Processing: s41598-022-12316-z.pdf
  ✓ Loaded 8 pages

Processing: s12911-022-01951-1.pdf
  ✓ Loaded 16 pages

Processing: s40620-023-01573-4.pdf


 17%|█▋        | 1/6 [1:02:48<5:14:03, 3768.60s/it]


  ✓ Loaded 17 pages

Processing: s40537-022-00657-5.pdf
  ✓ Loaded 19 pages

Processing: e002560.full.pdf
  ✓ Loaded 7 pages

Processing: 81878.pdf
  ✓ Loaded 14 pages

Total documents loaded: 81


In [17]:
all_pdf_documents[0]

Document(metadata={'producer': 'Adobe PDF Library 15.0; modified using iText® 5.3.5 ©2000-2012 1T3XT BVBA (SPRINGER SBM; licensed version)', 'creator': 'Springer', 'creationdate': '2022-05-19T09:40:50+05:30', 'crossmarkdomains[1]': 'springer.com', 'moddate': '2022-05-19T08:13:56+02:00', 'crossmarkmajorversiondate': '2010-04-23', 'subject': 'Scientific Reports, https://doi.org/10.1038/s41598-022-12316-z', 'author': 'Qiong Bai', 'title': 'Machine learning to predict end stage kidney disease in chronic kidney disease', 'crossmarkdomainexclusive': 'true', 'robots': 'noindex', 'doi': '10.1038/s41598-022-12316-z', 'crossmarkdomains[2]': 'springerlink.com', 'source': '/content/drive/MyDrive/Rag_Data/s41598-022-12316-z.pdf', 'total_pages': 8, 'page': 0, 'page_label': '1', 'source_file': 's41598-022-12316-z.pdf', 'file_type': 'pdf'}, page_content='1\nVol.:(0123456789)Scientific Reports |         (2022) 12:8377  | https://doi.org/10.1038/s41598-022-12316-z\nwww.nature.com/scientificreports\nMach

# **EMBEDDING AND VECTORDATA-BASE**

In [8]:
###Libraries
### sentence-transformers
### faiss cpu
### chromadb
#installation
!pip install sentence-transformers
!pip install faiss-cpu
!pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 21.3 MB/s eta 0

In [19]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")

    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")

    return split_docs

In [9]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
class EmbeddingManager:
  """ Handle Paper Embedding generation using Sentence Transformer"""
  def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
    """
    Initialize embedding manager.

    Args:
        model_name (str): Hugging face model name for sentence embeddings
    """
    self.model_name = model_name
    self.model = None
    self._load_model()

  def _load_model(self):
    """
    Load sentence transformer model.
    """
    try:
      print(f"Loading Sentence Transformer model: {self.model_name}")
      self.model = SentenceTransformer(self.model_name)
      print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
    except Exception as e:
      print(f"Error loading Sentence Transformer model: {e}")
      raise

  def generate_embeddings(self, texts: List[str]) -> List[np.ndarray]:
    """
    Generate embeddings for a list of texts.

    Args:
        texts (List[str]): List of texts to generate embeddings for.

    Returns:
        List[np.ndarray]: List of embeddings.
    """
    if not self.model:
      raise ValueError("Model not loaded. Call load_model() first.")

    print(f"Generating embeddings for {len(texts)} texts....")
    embeddings = self.model.encode(texts, show_progress_bar= True)
    print(f"Embeddings generated successfully. Shape: {embeddings.shape}")
    return embeddings

#Initialize the embeddings
embedding_manager = EmbeddingManager()
embedding_manager

Loading Sentence Transformer model: all-MiniLM-L6-v2


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Model loaded successfully. Embedding dimension: 384


In [22]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""

    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "/content/drive/MyDrive/Rag_Data/vector_store"):
        """
        Initialize the vector store

        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)

            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store

        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")

        print(f"Adding {len(documents)} documents to vector store...")
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []

        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)

            # Document content
            documents_text.append(doc.page_content)

            # Embedding
            embeddings_list.append(embedding.tolist())

        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                 documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 0


In [23]:
chunks=split_documents(all_pdf_documents)
chunks

Split 81 documents into 413 chunks

Example chunk:
Content: 1
Vol.:(0123456789)Scientific Reports |         (2022) 12:8377  | https://doi.org/10.1038/s41598-022-12316-z
www.nature.com/scientificreports
Machine learning to predict end 
stage kidney disease in c...
Metadata: {'producer': 'Adobe PDF Library 15.0; modified using iText® 5.3.5 ©2000-2012 1T3XT BVBA (SPRINGER SBM; licensed version)', 'creator': 'Springer', 'creationdate': '2022-05-19T09:40:50+05:30', 'crossmarkdomains[1]': 'springer.com', 'moddate': '2022-05-19T08:13:56+02:00', 'crossmarkmajorversiondate': '2010-04-23', 'subject': 'Scientific Reports, https://doi.org/10.1038/s41598-022-12316-z', 'author': 'Qiong Bai', 'title': 'Machine learning to predict end stage kidney disease in chronic kidney disease', 'crossmarkdomainexclusive': 'true', 'robots': 'noindex', 'doi': '10.1038/s41598-022-12316-z', 'crossmarkdomains[2]': 'springerlink.com', 'source': '/content/drive/MyDrive/Rag_Data/s41598-022-12316-z.pdf', 'total_pages': 8,

[Document(metadata={'producer': 'Adobe PDF Library 15.0; modified using iText® 5.3.5 ©2000-2012 1T3XT BVBA (SPRINGER SBM; licensed version)', 'creator': 'Springer', 'creationdate': '2022-05-19T09:40:50+05:30', 'crossmarkdomains[1]': 'springer.com', 'moddate': '2022-05-19T08:13:56+02:00', 'crossmarkmajorversiondate': '2010-04-23', 'subject': 'Scientific Reports, https://doi.org/10.1038/s41598-022-12316-z', 'author': 'Qiong Bai', 'title': 'Machine learning to predict end stage kidney disease in chronic kidney disease', 'crossmarkdomainexclusive': 'true', 'robots': 'noindex', 'doi': '10.1038/s41598-022-12316-z', 'crossmarkdomains[2]': 'springerlink.com', 'source': '/content/drive/MyDrive/Rag_Data/s41598-022-12316-z.pdf', 'total_pages': 8, 'page': 0, 'page_label': '1', 'source_file': 's41598-022-12316-z.pdf', 'file_type': 'pdf'}, page_content='1\nVol.:(0123456789)Scientific Reports |         (2022) 12:8377  | https://doi.org/10.1038/s41598-022-12316-z\nwww.nature.com/scientificreports\nMac

In [24]:
### Convert the text to embeddings
texts=[doc.page_content for doc in chunks]
## Generate the Embeddings
embeddings=embedding_manager.generate_embeddings(texts)
##store int he vector dtaabase
vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 413 texts....


Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Embeddings generated successfully. Shape: (413, 384)
Adding 413 documents to vector store...
Successfully added 413 documents to vector store
Total documents in collection: 413


In [25]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""

    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever

        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query

        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold

        Returns:
            List of dictionaries containing retrieved documents and metadata
            """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")

        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]

        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )

            # Process results
            retrieved_docs = []

            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]

                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance

                    if similarity_score >= score_threshold:
                      retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })

                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")

            return retrieved_docs

        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)

In [26]:
rag_retriever

In [32]:
rag_retriever.retrieve("What is a Chronic Kidney Disease?")

Retrieving documents for query: 'What is a Chronic Kidney Disease?'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts....


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Embeddings generated successfully. Shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_fa088aef_129',
  'content': '1103Journal of Nephrology (2023) 36:1101–1117 \n1 3\nIntroduction\nChronic Kidney Disease (CKD) is a state of progressive loss \nof kidney function ultimately resulting in the need for renal \nreplacement therapy (dialysis or transplantation) [1 ]. It is \ndefined as the presence of kidney damage or an estimated \nglomerular filtration rate less than 60\xa0ml/min per 1.73  m2, \npersisting for 3\xa0months or more [2]. CKD prevalence is grow-\ning worldwide, along with demographic and epidemiologi-\ncal transitions [3]. The implications of this disease are enor-\nmous for our society in terms of quality of life and the overall \nsustainability of national health systems. Worldwide, CKD \naccounted for 2,968,600 (1%) disability-adjusted life-years \nand 2,546,700 (1% to 3%) life-years lost in 2012 [4]. There-\nfore, it is of the utmost importance to assess how to promptly \nand adequately diagnose and treat patients with CKD.\nThe causes of CKD v

In [29]:
rag_retriever.retrieve("Best ML model for classifying Kidney Disease?")

Retrieving documents for query: 'Best ML model for classifying Kidney Disease?'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts....


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Embeddings generated successfully. Shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_99461e9a_194',
  'content': '1115Journal of Nephrology (2023) 36:1101–1117 \n1 3\n 36. Chen Z, Zhang X, Zhang Z (2016) Clinical risk assessment \nof patients with chronic kidney disease by using clinical data \nand multivariate models. Int Urol Nephrol 48(12):2069–2075. \nhttps:// doi. org/ 10. 1007/ s11255- 016- 1346-4\n 37. Rashed-Al-Mahfuz M, Haque A, Azad A et\xa0al (2021) Clinically \napplicable machine learning approaches to identify attributes \nof chronic kidney disease (CKD) for use in low-cost diagnostic \nscreening. IEEE J Transl Eng Health Med 9:4900511. https://  \ndoi. org/ 10. 1109/ JTEHM. 2021. 30736 29\n 38. Roth JA, Radevski G, Marzolini C et\xa0al (2021) Cohort-derived \nmachine learning models for individual prediction of chronic \nkidney disease in people living with human immunodeficiency \nvirus: a prospective multicenter cohort study. J Infect Dis \n224(7):1198–1208. https:// doi. org/ 10. 1093/ infdis/ jiaa2 36\n 39. Huang ML, Chou YC (2019) Combin